In [ ]:
%load_ext autoreload 
%autoreload 2
import benchml as bml
import numpy as np
log = bml.log

## Defining new models

In [ ]:
def build_linear():
    return bml.pipeline.Module(
        tag="Linear",
        transforms=[
            bml.transforms.ExttInput(),
            bml.transforms.CleanMatrix(
                inputs={
                    "X":"ExttInput.X"
                }),
            bml.transforms.LinearRegression(
                inputs={
                    "X": "CleanMatrix.X",
                    "y": "ExttInput.Y"
                })
        ],
        outputs={"y":"LinearRegression.y"})

def build_ridge():
    return bml.pipeline.Module(
        tag="Ridge",
        transforms=[
            bml.transforms.ExttInput(),
            bml.transforms.CleanMatrix(
                inputs={
                    "X":"ExttInput.X"
                }),
            bml.transforms.Ridge(
                inputs={
                    "X": "CleanMatrix.X",
                    "y": "ExttInput.Y"
                })
        ],
        hyper=bml.transforms.BayesianHyper(
            bml.transforms.Hyper({"Ridge.alpha": np.linspace(-7,7,15)}),
            convert={"Ridge.alpha": (lambda p: 10**p)}
        ),
        outputs={"y":"Ridge.y"})

def build_mp():
    return bml.pipeline.Module(
        tag="MarchenkoLinear",
        transforms=[
            bml.transforms.ExttInput(),
            bml.transforms.CleanMatrix(
                args={
                    "axis": 0, 
                    "std_threshold": 1e-10
                },
                inputs={
                    "X":"ExttInput.X"
                }),
            bml.transforms.MarchenkoPasturFilter(
                args={
                     "monomials": [1,2]
                },
                inputs={
                    "X":"CleanMatrix.X"
                }),
            bml.transforms.LinearRegression(
                inputs={
                    "X": "MarchenkoPasturFilter.X",
                    "y": "ExttInput.Y"
                })
        ],
        hyper=bml.transforms.GridHyper(
            bml.transforms.Hyper({"CleanMatrix.std_threshold": [1e-10, 0.5]}),
            bml.transforms.Hyper({"MarchenkoPasturFilter.monomials": [ [1], [2], [1,2] ]})
        ),
        outputs={"y":"LinearRegression.y"})

def build_rf():
    return bml.pipeline.Module(
        tag="RandomForest",
        transforms=[
            bml.transforms.ExttInput(),
            bml.transforms.RandomForestRegressor(
                tag="predictor",
                args={
                    "n_estimators": 10
                },
                inputs={"X": "ExttInput.X", "y": "ExttInput.Y"}),
        ],
        hyper=bml.transforms.GridHyper(
            bml.transforms.Hyper({"predictor.max_depth": [2,4,8]})
        ),
        broadcast={},
        outputs={"y": "predictor.y"})

def build_models():
    return [
        build_linear(),
        build_ridge(),
        build_rf(),
        build_mp()
    ]


## Listing transforms and models

In [ ]:
trafos = list(bml.transforms.get_all())
_ = [ print("%2d" % idx, t.__name__) for idx, t in enumerate(trafos) ]

In [ ]:
bml.models.list_all()

In [ ]:
model = bml.models.get("logd_delta_hybrid_gylm_krr").pop()
model.showHelpMessage()

## Inspecting a model

In [ ]:
model = build_mp()
print(model)

In [ ]:
args = model.compileArgs()
print("Args =")
_ = [ print("  %-40s = %-10s" % (k,v)) for k, v in args.items() ]

In [ ]:
model.showHelpMessage()

## Fitting a model

In [ ]:
def fit_evaluate_model(
        model, 
        dataset, 
        metrics=["mae", "r2", "rhop"],
        split=dict(method="random", n_splits=10, train_fraction=0.9)):
    accu = bml.Accumulator(metrics=metrics)
    with bml.stream(model, dataset) as stream:
        for idx, (train, test) in enumerate(stream.split(**split)):
            model.fit(train)
            output_train = model.map(train)
            output_test = model.map(test)
            accu.append("train", output_train["y"], train["ExttInput.Y"])
            accu.append("test", output_test["y"], test["ExttInput.Y"])
        print("Accumulated metrics:")
        res = accu.evaluateAll(log=bml.log, bootstrap=0)
    return model, res

In [ ]:
dataset = bml.load_dataset("ising.extt")
print(dataset)
model = build_mp()
model, res = fit_evaluate_model(model, dataset)
print(model)

In [ ]:
bml.splits.synchronize(0)
models = build_models()
timer = bml.utils.StagedTimer()
for m in models:
    print("Evaluate model '%s'" % m.tag)
    with timer.time(m.tag):
        model, res = fit_evaluate_model(m, dataset)
print("Wall times:")
timer.report(log)

## Hyperfitting and benchmarking

In [ ]:
def hyperfit_evaluate_model(
        model, 
        dataset, 
        accu,
        split=dict(method="random", n_splits=1, train_fraction=0.9),
        hypersplit=dict(method="random", n_splits=10, train_fraction=0.8)):
    with bml.stream(model, dataset) as stream:
        for idx, (train, test) in enumerate(stream.split(**split)):
            if model.hyper is not None:
                model.hyperfit(
                    stream=train,
                    split_args=hypersplit,
                    accu_args=dict(metric=accu.metrics[0]),
                    target="y",
                    target_ref="ExttInput.Y",
                    log=log)
            else:
                model.fit(stream=train)
            output_train = model.map(train)
            output_test = model.map(test)
            accu.append("%s_train" % model.tag, output_train["y"], train["ExttInput.Y"])
            accu.append("%s_test" % model.tag, output_test["y"], test["ExttInput.Y"])
    return res

In [ ]:
models = build_models()
accu = bml.Accumulator(metrics=["r2", "r2", "rhop"])
timer = bml.utils.StagedTimer()
for m in models:
    print("Evaluate model '%s'" % m.tag)
    with timer.time(m.tag):
        res = hyperfit_evaluate_model(m, dataset, accu)
print("Accumulated metrics:")
res = accu.evaluateAll(log=bml.log, bootstrap=0)

In [ ]:
timer.report(log)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plot
plot.style.use('seaborn-white')
fig, ax = plot.subplots(1, 4, sharex='col', sharey='row', figsize=(16,4))
for midx, m in enumerate(models):
    yp, yt = accu["%s_test" % m.tag]
    yp_train, yt_train = accu["%s_train" % m.tag]
    ax[midx].scatter(yp, yt, c='#0066ff')
    ax[midx].scatter(yp_train, yt_train, s=1.0, c='#333333')
    ax[midx].set_title(m.tag)

## Train and deploy

In [ ]:
model = build_mp()
with bml.stream(model, dataset) as stream:
    if model.hyper is not None:
        model.hyperfit(
            stream=stream,
            split_args=dict(method="kfold", k=10),
            accu_args=dict(metric="r2"),
            target="y",
            target_ref="ExttInput.Y",
            log=log)
    else:
        model.fit(stream)
bml.save("model.arch", model)

In [ ]:
model = bml.load("model.arch")
model.showHelpMessage()
with bml.stream(model, dataset) as stream:
    output = model.map(stream)
print(output)